In [4]:
import glob
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

In [13]:
def extract_feature(file_path):
    audio, sample_rate = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(audio, sr=sample_rate, n_mfcc=1)
    return mfccs

In [7]:
def dataset_generator():
    dataset = []
    normal_files = glob.glob("data/-6_dB_slider/slider/id_00/normal/*.wav")
    abnormal_files = glob.glob("data/-6_dB_slider/slider/id_00/abnormal/*.wav")
    for file in normal_files:
        features = extract_feature(file)
        dataset.append([features, 0])
        
    for file in abnormal_files:
        features = extract_feature(file)
        dataset.append([features, 1])
        
    train_data = dataset[:len(normal_files)-len(abnormal_files)]
    test_data = dataset[len(normal_files)-len(abnormal_files):]
    print("train data:",train_data.shape,"test data:",test_data.shape)
    return train_data, test_data

In [8]:
dataset_generator()

AttributeError: 'list' object has no attribute 'shape'

In [14]:
dataset = []
normal_files = glob.glob("data/-6_dB_slider/slider/id_00/normal/*.wav")
abnormal_files = glob.glob("data/-6_dB_slider/slider/id_00/abnormal/*.wav")
for file in normal_files:
    features = extract_feature(file)
    dataset.append([features, 0])

for file in abnormal_files:
    features = extract_feature(file)
    dataset.append([features, 1])

train_data = dataset[:len(normal_files)-len(abnormal_files)]
test_data = dataset[len(normal_files)-len(abnormal_files):]
print("train data:",train_data,"test data:",test_data)

train data: [[array([[-399.6518 , -391.64   , -390.6422 , -389.28693, -383.64453,
        -381.50797, -386.72742, -386.3299 , -383.40137, -383.92044,
        -383.4989 , -388.93738, -390.41528, -390.9067 , -389.4545 ,
        -386.05777, -381.06244, -380.75714, -388.21768, -385.14636,
        -381.42252, -383.4801 , -383.12527, -383.48328, -384.3038 ,
        -386.60303, -384.06842, -385.45105, -391.34454, -392.26273,
        -390.54083, -386.42484, -388.05362, -390.82712, -385.8589 ,
        -385.49524, -388.77054, -390.88495, -388.44504, -383.58353,
        -385.7951 , -392.31314, -391.80856, -392.55145, -393.76047,
        -390.69324, -387.5851 , -386.33618, -385.08636, -384.21716,
        -389.75037, -388.91037, -386.13846, -385.67108, -389.8569 ,
        -387.2036 , -388.05017, -391.78668, -394.834  , -391.41864,
        -390.71222, -387.55722, -385.66806, -384.65   , -381.94736,
        -386.6922 , -386.54028, -387.80093, -392.91318, -391.75406,
        -388.32938, -389.73727, -3

In [15]:
len(train_data)

15

In [16]:
len(test_data)

10

In [25]:
X_train = np.array(train_data[][0]).reshape(-1,1)

/tmp/ipykernel_51290/1946259483.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(train_data[:][0]).reshape(-1,1)


In [37]:
train_data[0][0]

array([[-399.6518 , -391.64   , -390.6422 , -389.28693, -383.64453,
        -381.50797, -386.72742, -386.3299 , -383.40137, -383.92044,
        -383.4989 , -388.93738, -390.41528, -390.9067 , -389.4545 ,
        -386.05777, -381.06244, -380.75714, -388.21768, -385.14636,
        -381.42252, -383.4801 , -383.12527, -383.48328, -384.3038 ,
        -386.60303, -384.06842, -385.45105, -391.34454, -392.26273,
        -390.54083, -386.42484, -388.05362, -390.82712, -385.8589 ,
        -385.49524, -388.77054, -390.88495, -388.44504, -383.58353,
        -385.7951 , -392.31314, -391.80856, -392.55145, -393.76047,
        -390.69324, -387.5851 , -386.33618, -385.08636, -384.21716,
        -389.75037, -388.91037, -386.13846, -385.67108, -389.8569 ,
        -387.2036 , -388.05017, -391.78668, -394.834  , -391.41864,
        -390.71222, -387.55722, -385.66806, -384.65   , -381.94736,
        -386.6922 , -386.54028, -387.80093, -392.91318, -391.75406,
        -388.32938, -389.73727, -393.76843, -392

In [26]:
y_train = np.array(train_data[:][1]).reshape(-1,1)

/tmp/ipykernel_51290/262586237.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array(train_data[:][1]).reshape(-1,1)


In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [27]:
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(X_train, y_train)

ValueError: setting an array element with a sequence.